In [11]:
import json
with open('train_data.json', 'r') as f:
    data = f.read()
    data = json.loads(data)

In [2]:
print(data[0])

{'movie_id': 92575, 'review': '종합 평점은 4점 드립니다.', 'rating': 4, 'date': '15.08.26'}


In [3]:
review = [dic['review'] for dic in data]
rating = [dic['rating'] for dic in data]

sep = int(len(review)*0.9)

train_review = review[:sep]
train_rating = rating[:sep]
val_review = review[sep:]
val_rating = rating[sep:]

In [4]:
import pandas as pd
from konlpy.tag import Twitter
from gensim.models.word2vec import Word2Vec
from random import shuffle

pos_tagger = Twitter()

def tokenize(doc):
        return ['/'.join(t) for t in pos_tagger.pos(str(doc), norm=True, stem=True)]

def make_wv(data,tag=True,using_tag = ['Noun','Verb','Adjective'],size=100,window=5,min_count=50,workers=4):
        
    if tag is True:
        try:
            data_tag = [tokenize(x) for x in data]
        except:
            print(data)
            raise
    else:
        data_tag = x
    
    def tag_filter(x):
        out = []
        for t in x:
            if t.split('/')[1] in using_tag:
                out.append(t)
        return out
    
    data_filter = [tag_filter(x) for x in data_tag]
    model = Word2Vec(data_filter, size=size, window=window, min_count=min_count, iter=10, workers=workers)
    return model
    
    #wv_df =pd.DataFrame(model.wv[model.wv.vocab])
    
    #return pd.concat([pd.Series(list(model.wv.vocab.keys())),wv_df],axis=1)

In [5]:
wv = make_wv(train_review) # train, val 나눠서 해야함

In [6]:
from konlpy.utils import pprint
pprint(wv.most_similar(positive=tokenize(u'여배우 남자'), negative=tokenize(u'배우'), topn=1))

[('여자/Noun', 0.8290437459945679)]


/Users/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [7]:
voca = list(wv.wv.vocab.keys())

In [8]:
embed = wv.wv[wv.wv.vocab]
voca = list(wv.wv.vocab.keys())

In [9]:
def sen2vec(sentence):
    token = tokenize(sentence)
    vec = [0 for i in range(100)]
    count = 0
    for word in token:
        if word in voca:
            vec = [i+j for i, j in zip(vec, embed[voca.index(word)])]
    if count == 0:
        return vec
    else:
        return [i/count for i in vec]

In [10]:
train_review_vec = [sen2vec(i) for i in train_review]
val_review_vec = [sen2vec(i) for i in val_review]

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [47]:
def labeling(x):
    if x >= 1 and x <=3:
        return 'NEG'
    elif x >= 4 and x <= 7:
        return 'NEU'
    else:
        return 'POS'

In [55]:
regr = RandomForestRegressor()
regr.fit(train_review_vec, train_rating)

val_label = list(map(labeling, val_rating))
val_pred_label = list(map(labeling, regr.predict(val_review_vec)))

sum([i==j for i, j in zip(val_label, val_pred_label)])/len(val_pred_label)

0.6314